
# WEB SCRAPING - Get Stock Headlines - [Using: Selenium, Chron ]

##  <font color='red' size="4"> This web scraping will show new headlines on stocks for last 24 hours </font></b> 

### Table of content:
      1.Load and Install libraries
      2.get data step by step
      3.create schedule by Linux & Windows

<b> <font color='Blue' size="4"> 1. Load and install libraries</font></b> 

In [252]:
#install libraries
'''
! pip install selenium
conda install -c conda-forge python-chromedriver-binary 
'''

'\n! pip install selenium\nconda install -c conda-forge python-chromedriver-binary \n'

In [253]:
#import libraries

from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep 
import requests
import urllib
import pandas as pd

# create webdriver object
#driver = webdriver.Chrome(ChromeDriverManager().install())


<b> <font color='Blue' size="4"> 2. Get data step by step</font></b> 

### select stock

In [254]:
stock = 'tesla'

### Create connection with browser 

<b> <font color='grey' size="4"> Connect to google</font></b> 

In [255]:
driver = webdriver.Chrome('/home/idan/chromedriver_linux64/chromedriver')
driver.get('https://www.google.com/')

<b> <font color='grey' size="4"> submit stock name to server</font></b> 

In [256]:
search_bar = driver.find_element_by_xpath('//input[@class= "gLFyf gsfi"]')
search_bar.send_keys(stock)
search_bar.submit()
sleep(5)

<b> <font color='grey' size="4"> Click on 'news' tag</font></b> 

In [258]:
select_news = driver.find_element_by_xpath('//div[@id = "hdtb-msb-vis"]//div[@class = "hdtb-mitem hdtb-imb"]//following-sibling::div[@class = "hdtb-mitem hdtb-imb"]//a[@class= "q qs"]')
select_news.click()
sleep(2)

<b> <font color='grey' size="4"> click on 'Tools' button</font></b> 

In [259]:
tools_button = driver.find_element_by_xpath('//a[@id= "hdtb-tls"]')
tools_button.click()
sleep(2)

<b> <font color='grey' size="4"> select time search bar</font></b> 

In [260]:
select_time_search_bar = driver.find_element_by_xpath('//div[@id = "hdtbMenus"]//div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]')
select_time_search_bar.click()
sleep(2)

<b> <font color='grey' size="4"> select last 24 hours</font></b> 

In [261]:
last_day = driver.find_element_by_xpath('//ul[@class = "hdtbU hdtb-mn-o"]//li[@id= "qdr_d"]')
last_day.click()
sleep(2)

<b> <font color='grey' size="4"> Get current url</font></b> 

In [262]:
driver.current_url

'https://www.google.com/search?q=tesla&tbm=nws&source=lnt&tbs=qdr:d&sa=X&ved=0ahUKEwjR-6PRrdzrAhUFRhUIHekgD08QpwUIJA&biw=781&bih=872&dpr=1'

<b> <font color='grey' size="4"> Show Headlines in first page</font></b> 

In [316]:
articles = driver.find_elements_by_xpath('//div[@class= "dbsr"]//span[@dir="ltr"]')
for article in articles :
    print(article.text)

CNN
Tesla shares suffer their worst day ever
CNBC
Tesla is a long-term buy that's 'just getting started,' says ...
Financial Times
Tesla shares tumble in biggest-ever decline
The Australian Financial Review
Tesla's worst-ever fall leaves investors shaken
MarketWatch
Did you miss out on Tesla’s big run? There are still good alternatives to play the electric-car revolution
Los Angeles Times
Tesla stock falls 21% in worst-ever trading day
Benzinga
Tesla Model Y A 'Thought Through' Car, Volkswagen CEO Showers Praise After Test Drive
Teslarati
Tesla bull ARK Invest is loading up on TSLA once more amid ongoing dip
CNBC
Tesla is a ‘tech on wheels’ company, and that gives it room to run, trader says
CleanTechnica
Anti-Tesla Talking Point Crushed By UK Survey — Tesla Model 3 Is Reliability King


<b> <font color='grey' size="4"> Get Headlines From All pages by loop</font></b> 

In [248]:
%%time

class get_headlines :
    
      def __init__ (self,stock_name):
            self.stock = stock_name
            self.articles = []
            self.website = []
            self.page = []
            self.table = []
            
      def connect_to_server(self):
            self.driver = webdriver.Chrome('/home/idan/chromedriver_linux64/chromedriver')
            self.driver.get('https://www.google.com/')
            self.search_bar = self.driver.find_element_by_xpath('//input[@class= "gLFyf gsfi"]')
            self.search_bar.send_keys(self.stock)
            self.search_bar.submit()
            sleep(5)
            self.select_news = self.driver.find_element_by_xpath('//div[@id = "hdtb-msb-vis"]//div[@class = "hdtb-mitem hdtb-imb"]//following-sibling::div[@class = "hdtb-mitem hdtb-imb"]//a[@class= "q qs"]')
            self.select_news.click()
            sleep(2)
            self.tools_button = self.driver.find_element_by_xpath('//a[@id= "hdtb-tls"]')
            self.tools_button.click()
            sleep(2)
            self.select_time_search_bar = self.driver.find_element_by_xpath('//div[@id = "hdtbMenus"]//div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]')
            self.select_time_search_bar.click()
            sleep(2)
            self.last_day = self.driver.find_element_by_xpath('//ul[@class = "hdtbU hdtb-mn-o"]//li[@id= "qdr_d"]')
            self.last_day.click()
            sleep(2)

    
      def articles_per_page (self):  
           self.tag = self.driver.find_elements_by_xpath('//div[@class= "dbsr"]//span[@dir="ltr"]')
           i=0
           while i <= len(self.tag):      
                self.website.append(self.tag[i].text)
                i+=1
                self.articles.append(self.tag[i].text)
                i+=1
                if i >= len(self.tag) :
                    break
        
           self.table = pd.DataFrame(list(zip(self.website,self.articles)),columns= ['website','headline'])   
    
      
      def page_number (self):
         self.connect_to_server()
         self.pages_batch = self.driver.find_elements_by_tag_name('td')
         self.page_index = 1 
         self.last_page = int(self.pages_batch[-2].text)

         while self.page_index <= self.last_page :
              #print(f'\n\npage {self.page_index}')
              self.articles_per_page()
              
              self.page = self.driver.find_elements_by_xpath('//span[@dir ="rtl"]')
              self.page[-1].click()
              sleep(5)
              self.pages_batch = self.driver.find_elements_by_tag_name('td')
              self.last_page = int(self.pages_batch[-2].text)  
              self.page_index +=1 
              self.previous_page = int(self.pages_batch[-3].text)  
              #print(f'\\npage_index={self.page_index},previous_page={self.previous_page},last_page={self.last_page}')
              
              if self.last_page < self.previous_page :
                 self.driver.close()
                 print('\n\nFinished !')
                 break
         
         print('\n\nFinished :)')
         self.driver.close()
       
      def output (self) :
          return self.table 
            
            
                
stock = 'tesla'     
    
if __name__ == '__main__' :
   activate = get_headlines(stock)
   activate.page_number() 
   headlines = activate.output()    





page 1


,website,headline
0,CNN,4 insane things just happened to Tesla
1,Los Angeles Times,Tesla stock falls 21% in worst-ever trading day
2,CNBC,Tesla is a long-term buy that's 'just getting ...
3,The Australian Financial Review,Tesla's worst-ever fall leaves investors shaken
4,Benzinga,"Tesla Model Y A 'Thought Through' Car, Volkswa..."
5,Teslarati,Tesla bull ARK Invest is loading up on TSLA on...
6,CNBC,"Tesla is a ‘tech on wheels’ company, and that ..."
7,CleanTechnica,Anti-Tesla Talking Point Crushed By UK Survey ...
8,TechCrunch,Panasonic to expand battery capacity at Tesla ...
9,Bloomberg,Call That a Correction? Tesla Tests the Lexicon


\npage_index=2,previous_page=10,last_page=11


page 2


,website,headline
0,CNN,4 insane things just happened to Tesla
1,Los Angeles Times,Tesla stock falls 21% in worst-ever trading day
2,CNBC,Tesla is a long-term buy that's 'just getting ...
3,The Australian Financial Review,Tesla's worst-ever fall leaves investors shaken
4,Benzinga,"Tesla Model Y A 'Thought Through' Car, Volkswa..."
5,Teslarati,Tesla bull ARK Invest is loading up on TSLA on...
6,CNBC,"Tesla is a ‘tech on wheels’ company, and that ..."
7,CleanTechnica,Anti-Tesla Talking Point Crushed By UK Survey ...
8,TechCrunch,Panasonic to expand battery capacity at Tesla ...
9,Bloomberg,Call That a Correction? Tesla Tests the Lexicon


\npage_index=3,previous_page=11,last_page=12


page 3


...

\npage_index=17,previous_page=17,last_page=18


page 17


,website,headline
0,CNN,4 insane things just happened to Tesla
1,Los Angeles Times,Tesla stock falls 21% in worst-ever trading day
2,CNBC,Tesla is a long-term buy that's 'just getting ...
3,The Australian Financial Review,Tesla's worst-ever fall leaves investors shaken
4,Benzinga,"Tesla Model Y A 'Thought Through' Car, Volkswa..."
5,Teslarati,Tesla bull ARK Invest is loading up on TSLA on...
6,CNBC,"Tesla is a ‘tech on wheels’ company, and that ..."
7,CleanTechnica,Anti-Tesla Talking Point Crushed By UK Survey ...
8,TechCrunch,Panasonic to expand battery capacity at Tesla ...
9,Bloomberg,Call That a Correction? Tesla Tests the Lexicon


\npage_index=18,previous_page=17,last_page=18


page 18


,website,headline
0,CNN,4 insane things just happened to Tesla
1,Los Angeles Times,Tesla stock falls 21% in worst-ever trading day
2,CNBC,Tesla is a long-term buy that's 'just getting ...
3,The Australian Financial Review,Tesla's worst-ever fall leaves investors shaken
4,Benzinga,"Tesla Model Y A 'Thought Through' Car, Volkswa..."
5,Teslarati,Tesla bull ARK Invest is loading up on TSLA on...
6,CNBC,"Tesla is a ‘tech on wheels’ company, and that ..."
7,CleanTechnica,Anti-Tesla Talking Point Crushed By UK Survey ...
8,TechCrunch,Panasonic to expand battery capacity at Tesla ...
9,Bloomberg,Call That a Correction? Tesla Tests the Lexicon


\npage_index=19,previous_page=17,last_page=18


Finished :)
CPU times: user 1.05 s, sys: 26.6 ms, total: 1.07 s
Wall time: 2min 41s


<b> <font color='grey' size="4"> Show Output</font></b> 

In [ ]:
display(headlines)

  
<b> <font color='Blue' size="4"> 3.1 create schedule by Chron</font></b> 

In [ ]:
1. open terminal
2. emacs (for emacs editor)
3. insert in text editor : * 0 * * * /usr/bin/python3 /home/idan/Desktop/stock_WebScraping.py
4. save by name task_scheduler 
5. crostab task_scheduler.txt

<b> <font color='Blue' size="4"> 3.2 create schedule by Windows</font></b> 

In [ ]:
1. create batch(text) file by notepad as follow:
  "C:\Users\Home\all\python\directories\python.exe" "C:/Users/python/script/directories/stock_WebScraping.py"

2. save notepad file as follow:
    notepad_name.bat
3. scroll and select .txt

4. open task scheduler => Activation Tab => Script folder: 
   insert C:\Users\Home\notepad\directory\notepad_name.bat
   
5. define your schedule